# New Instance CMR

In [1]:
import numpy as np
from numba import float64, int32, boolean
from numba.experimental import jitclass

icmr_spec = [
    ('item_count', int32), 
    ('encoding_drift_rate', float64),
    ('start_drift_rate', float64),
    ('recall_drift_rate', float64),
    ('delay_drift_rate', float64),
    ('shared_support', float64),
    ('item_support', float64),
    ('learning_rate', float64),
    ('primacy_scale', float64),
    ('primacy_decay', float64),
    ('stop_probability_scale', float64),
    ('stop_probability_growth', float64),
    ('choice_sensitivity', float64),
    ('context_sensitivity', float64),
    ('feature_sensitivity', float64),
    ('context', float64[::1]),
    ('start_context_input', float64[::1]),
    ('delay_context_input', float64[::1]),
    ('preretrieval_context', float64[::1]),
    ('recall', int32[::1]),
    ('retrieving', boolean),
    ('recall_total', int32),
    ('item_weighting', float64[::1]),
    ('context_weighting', float64[::1]),
    ('all_weighting', float64[::1]),
    ('probabilities', float64[::1]),
    ('memory', float64[:,::1]),
    ('encoding_index', int32),
    ('items', float64[:,::1]),
    ('norm', float64[::1]),
]

#int32 = int


## Working Model

In [2]:


@jitclass(icmr_spec)
class Instance_CMR:

    def __init__(self, item_count, presentation_count, parameters):

        # store initial parameters
        self.item_count = item_count
        self.encoding_drift_rate = parameters['encoding_drift_rate']
        self.delay_drift_rate = parameters['delay_drift_rate']
        self.start_drift_rate = parameters['start_drift_rate']
        self.recall_drift_rate = parameters['recall_drift_rate']
        self.shared_support = parameters['shared_support']
        self.item_support = parameters['item_support']
        self.learning_rate = parameters['learning_rate']
        self.primacy_scale = parameters['primacy_scale']
        self.primacy_decay = parameters['primacy_decay']
        self.stop_probability_scale = parameters['stop_probability_scale']
        self.stop_probability_growth = parameters['stop_probability_growth']
        self.choice_sensitivity = parameters['choice_sensitivity']
        self.context_sensitivity = parameters['context_sensitivity']
        self.feature_sensitivity = parameters['feature_sensitivity']
        
        # at the start of the list context is initialized with a state 
        # orthogonal to the pre-experimental context associated with the set of items
        self.context = np.zeros(item_count + 1)
        self.context[0] = 1
        self.preretrieval_context = self.context
        self.recall = np.zeros(item_count) # recalls has at most `item_count` entries
        self.retrieving = False
        self.recall_total = 0

        # predefine activation weighting vectors
        self.item_weighting = np.ones(item_count+presentation_count)
        self.context_weighting = np.ones(item_count+presentation_count)
        self.item_weighting[item_count:] = self.learning_rate
        self.context_weighting[item_count:] = \
            self.primacy_scale * np.exp(-self.primacy_decay * np.arange(presentation_count)) + 1
        self.all_weighting = self.item_weighting * self.context_weighting

        # preallocate for outcome_probabilities
        self.probabilities = np.zeros((item_count + 1))

        # initialize memory
        # we now conceptualize it as a pairing of two stores Mfc and Mcf respectively
        # representing feature-to-context and context-to-feature associations
        mfc = np.eye(item_count, item_count + 1, 1) * (1 - self.learning_rate)
        mcf = np.ones((item_count, item_count)) * self.shared_support
        for i in range(item_count):
            mcf[i, i] = self.item_support
        mcf = np.hstack((np.zeros((item_count, 1)), mcf))
        self.memory = np.zeros((item_count + presentation_count, item_count * 2 + 2))
        self.memory[:item_count,] = np.hstack((mfc, mcf))
        self.encoding_index = item_count
        self.items = np.eye(item_count, item_count + 1, 1)

    def experience(self, experiences):

        for i in range(len(experiences)):
            self.memory[self.encoding_index, :self.item_count+1] = experiences[i]
            self.update_context(self.encoding_drift_rate, self.memory[self.encoding_index])
            self.memory[self.encoding_index, self.item_count+1:] = self.context
            self.encoding_index += 1

    def update_context(self, drift_rate, experience=None):

        # first pre-experimental or initial context is retrieved
        if experience is not None:
            context_input = self.echo(experience)[self.item_count + 1:]
            context_input = context_input / np.sqrt(np.sum(np.square(context_input))) # norm to length 1
        else:
            context_input = np.zeros((self.item_count+1))
            context_input[0] = 1

        # updated context is sum of context and input, modulated by rho to have len 1 and some drift_rate
        rho = np.sqrt(1 + np.square(drift_rate) * (np.square(self.context * context_input) - 1)) - (
                drift_rate * (self.context * context_input))
        self.context = (rho * self.context) + (drift_rate * context_input)

    def echo(self, probe):

        return np.dot(self.activations(probe), self.memory[:self.encoding_index])

    def activations(self, probe):

        # computes and cubes similarity value to find activation for each trace in memory
        activation = np.dot(self.memory[:self.encoding_index], probe) / (
            np.sqrt(np.sum(np.square(self.memory[:self.encoding_index]), axis=1)) * np.sqrt(
                np.sum(np.square(probe))))

        # weight activations based on whether probe contains item or contextual features or both
        if np.any(probe[:self.item_count + 1]):
            if np.any(probe[self.item_count + 1:]):
                # both mfc and mcf weightings, see below
                activation *= self.all_weighting[:self.encoding_index]
            else:
                # mfc weightings - scale by gamma for each experimental trace
                activation *= self.item_weighting[:self.encoding_index]
            activation = np.power(activation, self.context_sensitivity)
        else:
            # mcf weightings - scale by primacy/attention function based on experience position
            activation *= self.context_weighting[:self.encoding_index]
            if self.feature_sensitivity != 1.0:
                activation = np.power(activation, self.feature_sensitivity)
            else:
                activation = np.power(activation, self.context_sensitivity)
            
        return activation + 10e-7

    def outcome_probabilities(self):

        activation_cue = np.hstack((np.zeros(self.item_count + 1), self.context))
        echo = self.echo(activation_cue)[1:self.item_count+1]
        echo = np.power(echo, self.choice_sensitivity)
        
        self.probabilities = np.zeros((self.item_count + 1))
        self.probabilities[0] = min(self.stop_probability_scale * np.exp(
            self.recall_total * self.stop_probability_growth), 1.0 - (self.item_count * 10e-7))

        if self.probabilities[0] < 1:
            for already_recalled_item in self.recall[:self.recall_total]:
                echo[int(already_recalled_item)] = 0
        self.probabilities[1:] = (1-self.probabilities[0]) * echo / np.sum(echo)
        
        return self.probabilities

    def free_recall(self, steps=None):

        # some pre-list context is reinstated before initiating recall
        if not self.retrieving:
            self.recall = np.zeros(self.item_count)
            self.recall_total = 0
            self.preretrieval_context = self.context
            self.update_context(self.start_drift_rate)
            self.retrieving = True
            
        # number of items to retrieve is infinite if steps is unspecified
        if steps is None:
            steps = self.item_count - self.recall_total
        steps = self.recall_total + steps

        # at each recall attempt
        while self.recall_total < steps:

            # the current state of context is used as a retrieval cue to 
            # attempt recall of a studied item compute outcome probabilities 
            # and make choice based on distribution
            outcome_probabilities = self.outcome_probabilities(
                np.hstack((np.zeros(self.item_count + 1), self.context)))
            if np.any(outcome_probabilities[1:]):
                choice = np.sum(
                    np.cumsum(outcome_probabilities) < np.random.rand())
            else:
                choice = 0

            # resolve and maybe store outcome
            # we stop recall if no choice is made (0)
            if choice == 0:
                self.retrieving = False
                self.context = self.preretrieval_context
                break
            self.recall[self.recall_total] = choice - 1
            self.recall_total += 1
            self.update_context(self.recall_drift_rate,
                                np.hstack((self.items[choice - 1], 
                                           np.zeros(self.item_count + 1))))
        return self.recall[:self.recall_total]
    
    def force_recall(self, choice=None):

        if not self.retrieving:
            self.recall = np.zeros(self.item_count)
            self.recall_total = 0
            self.preretrieval_context = self.context
            self.update_context(self.start_drift_rate)
            self.retrieving = True

        if choice is None:
            pass
        elif choice > 0:
            self.recall[self.recall_total] = choice - 1
            self.recall_total += 1
            self.update_context(
                self.recall_drift_rate, 
                np.hstack((self.items[choice - 1], 
                           np.zeros(self.item_count + 1))))
        else:
            self.retrieving = False
            self.context = self.preretrieval_context
        return self.recall[:self.recall_total]

## Fits

In [3]:
import numpy as np
from numba import njit, prange

@njit(fastmath=True, nogil=True, parallel=True)
def murdock_data_likelihood(data_to_fit, item_counts, model_class, parameters):

    result = 0.0
    for i in prange(len(item_counts)):
        item_count = item_counts[i]
        trials = data_to_fit[i]
        likelihood = np.ones((len(trials), item_count))

        model = model_class(item_count, item_count, parameters)
        model.experience(model.items)

        for trial_index in range(len(trials)):
            trial = trials[trial_index]

            model.force_recall()
            for recall_index in range(len(trial) + 1):

                # identify index of item recalled; if zero then recall is over
                if recall_index == len(trial) and len(trial) < item_count:
                    recall = 0
                else:
                    recall = trial[recall_index]

                # store probability of and simulate recall of indexed item 
                likelihood[trial_index, recall_index] = model.outcome_probabilities()[recall]
                
                if recall == 0:
                    break
                model.force_recall(recall)

            # reset model to its pre-retrieval (but post-encoding) state
            model.force_recall(0)
            
        result -= np.sum(np.log(likelihood))

    return result

In [4]:
from numba.typed import Dict
from numba.core import types

def murdock_objective_function(
    data_to_fit, item_counts, model_class, fixed_parameters, free_parameters):
    """
    Configures cmr_likelihood for search over specified free/fixed parameters.
    """

    parameters = Dict.empty(key_type=types.unicode_type, value_type=types.float64)
    for name, value in fixed_parameters.items():
        parameters[name] = value
    
    def objective_function(x):
        for i in range(len(free_parameters)):
            parameters[free_parameters[i]] = x[i]
        return murdock_data_likelihood(data_to_fit, item_counts, model_class, parameters)

    return objective_function

In [5]:
from numba.typed import Dict, List
from numba.core import types
from numba import njit
from compmemlearn.datasets import prepare_murdock1962_data

murd_trials0, murd_events0, murd_length0 = prepare_murdock1962_data('../../data/MurdData_clean.mat', 0)
murd_trials1, murd_events1, murd_length1 = prepare_murdock1962_data('../../data/MurdData_clean.mat', 1)
murd_trials2, murd_events2, murd_length2 = prepare_murdock1962_data('../../data/MurdData_clean.mat', 2)
free_parameters = [
    'encoding_drift_rate',
    'start_drift_rate',
    'recall_drift_rate',
    'shared_support',
    'item_support',
    'learning_rate',
    'primacy_scale',
    'primacy_decay',
    'stop_probability_scale',
    'stop_probability_growth',
#    'choice_sensitivity',
    'context_sensitivity',
#    'feature_sensitivity'
]

fit_values = np.array([7.25392222e-01, 2.22044605e-16, 9.04899967e-01, 1.16637532e-05,
       1.35117327e-03, 7.14751007e-03, 6.26753336e+00, 1.73084858e+00,
       3.05537134e-02, 3.38091912e-01, 1.54964510e+00])

fit_values = np.array([7.66747592e-01, 1.68847562e-03, 8.46922217e-01, 4.25187058e-03,
       1.00000000e+00, 2.07232433e-01, 8.68402303e+00, 5.53375476e+01,
       2.40297656e-02, 2.61885572e-01, 1.29423172e+00])

icmr_parameters = Dict.empty(key_type=types.unicode_type, value_type=types.float64)
for i in range(len(fit_values)):
    icmr_parameters[free_parameters[i]] = fit_values[i]
icmr_parameters['choice_sensitivity'] = 1.0
icmr_parameters['feature_sensitivity'] = 1.0
icmr_parameters['delay_drift_rate'] = 0.00001

In [ ]:
@njit(fastmath=True, nogil=True)
def init_icmr(item_count, presentation_count, parameters):
    return Instance_CMR(item_count, presentation_count, parameters)

murdock_data_likelihood(List([murd_trials0, murd_trials1, murd_trials2]), List([murd_length0,murd_length1, murd_length2]), init_icmr, icmr_parameters)

80350.59615655807

```
87081.77648 -> 97020.09199
```

In [ ]:
%%timeit
murdock_data_likelihood(List([murd_trials0, murd_trials1, murd_trials2]), List([murd_length0,murd_length1, murd_length2]),init_icmr, icmr_parameters)

4.2 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


```
350ms -> 216ms -> 148ms
3.82s
```

## Profile

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f murdock_data_likelihood murdock_data_likelihood(List([murd_trials0, murd_trials1, murd_trials2]), List([murd_length0,murd_length1, murd_length2]), init_icmr, icmr_parameters)

Timer unit: 1e-07 s

Total time: 6.63313 s

Could not find file C:\Users\gunnj\AppData\Local\Temp/ipykernel_19608/565983307.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           
     6                                           
     7         1         15.0     15.0      0.0  
     8         4         97.0     24.2      0.0  
     9         3        467.0    155.7      0.0  
    10         3        222.0     74.0      0.0  
    11         3        969.0    323.0      0.0  
    12                                           
    13         3      11839.0   3946.3      0.0  
    14         3      94619.0  31539.7      0.1  
    15                                           
    16      3603      19025.0      5.3      0.0  
    17      3600      28143.0      7.8      0.0  
    18           

`outcome_probabilities` and `force_recall` are the bottlenecks. Can I get the speed closer to regular CMR?

In [ ]:
def fake_data_likelihood(data_to_fit, item_counts, model_class, parameters):

    result = 0.0
    for i in prange(len(item_counts)):
        item_count = item_counts[i]
        trials = data_to_fit[i]
        likelihood = np.ones((len(trials), item_count))

        model = model_class(item_count, item_count, parameters)
        model.experience(model.items)

        for trial_index in range(len(trials)):
            trial = trials[trial_index]

            model.force_recall()
            for recall_index in range(len(trial) + 1):

                # identify index of item recalled; if zero then recall is over
                if recall_index == len(trial) and len(trial) < item_count:
                    recall = 0
                else:
                    recall = trial[recall_index]

                # store probability of and simulate recall of indexed item 
                return model

test = fake_data_likelihood(List([murd_trials0, murd_trials1, murd_trials2]), List([murd_length0,murd_length1, murd_length2]), init_icmr, icmr_parameters)

activation_cue = np.hstack((np.zeros(test.item_count + 1), test.context))
test.activations(activation_cue)

array([1.31485548e-06, 1.31729989e-06, 1.32090451e-06, 1.32621364e-06,
       1.33403417e-06, 1.34557333e-06, 1.36265751e-06, 1.38808834e-06,
       1.42623777e-06, 1.48406592e-06, 1.57289773e-06, 1.71156990e-06,
       1.93205791e-06, 2.28959237e-06, 2.88091177e-06, 3.87732019e-06,
       5.58487704e-06, 8.55480456e-06, 1.37879625e-05, 2.31177823e-05,
       7.59444482e-02, 2.33180846e-02, 1.38677434e-02, 1.36710923e-02,
       1.50261962e-02, 1.64806246e-02, 1.78685323e-02, 1.93059832e-02,
       2.09882105e-02, 2.31764405e-02, 2.62415032e-02, 3.07486748e-02,
       3.76055166e-02, 4.83225202e-02, 6.54788672e-02, 9.35614591e-02,
       1.40485255e-01, 2.20365615e-01, 3.58614468e-01, 6.01400581e-01])

In [ ]:
test.items[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [ ]:
%lprun -f test.outcome_probabilities test.outcome_probabilities()

Timer unit: 1e-07 s

Total time: 0.0004682 s

Could not find file C:\Users\gunnj\AppData\Local\Temp/ipykernel_19608/611620248.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
   109                                           
   110                                           
   111         2        772.0    386.0     16.5  
   112         1         57.0     57.0      1.2  
   113         1       3479.0   3479.0     74.3  
   114         1         26.0     26.0      0.6  
   115                                           
   116         1         25.0     25.0      0.5  
   117         3         56.0     18.7      1.2  
   118         2         14.0      7.0      0.3  
   119                                           
   120         1         17.0     17.0      0.4  
   121         1         32.0     32.0      0.7  
   122         

## ...

In [ ]:
item_count = 20

np.shape(np.eye(1, 2*(item_count+2), item_count+2).flatten()[item_count+2:])

(22,)

In [ ]:
np.shape(np.zeros(item_count + 1))

(21,)

In [ ]:
learning_rate = .7

np.eye(item_count, item_count + 2, 1) * (1 - learning_rate)

array([[0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 

In [ ]:
np.eye(item_count, item_count + 1, 1) * .3

array([[0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0

In [ ]:
np.hstack((np.eye(item_count, item_count + 2, 1), np.zeros(
            (item_count, item_count+2))))[:, :item_count+2]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,